In [1]:
!pip install rdflib boto3 gremlinpython arelle  aiohttp 

In [2]:
!ls -l

total 5852
-rw-rw-r-- 1 ec2-user ec2-user   23208 May 29 00:47 Amazon_XBRL_to_Neptune_GraphDB.ipynb
-rw-rw-r-- 1 ec2-user ec2-user  186788 May 29 00:30 amzn-20241231_cal.xml
-rw-rw-r-- 1 ec2-user ec2-user  382312 May 29 00:30 amzn-20241231_def.xml
-rw-rw-r-- 1 ec2-user ec2-user 1900279 May 29 00:30 amzn-20241231_htm.xml
-rw-rw-r-- 1 ec2-user ec2-user 1025798 May 29 00:30 amzn-20241231_lab.xml
-rw-rw-r-- 1 ec2-user ec2-user  703884 May 29 00:30 amzn-20241231_pre.xml
-rw-rw-r-- 1 ec2-user ec2-user   85177 May 29 00:30 amzn-20241231.xsd
-rw-rw-r-- 1 ec2-user ec2-user 1654667 May 29 00:30 amzn_facts.json
drwx------ 2 root     root       16384 May 28 03:07 lost+found
drwxrwxr-x 7 ec2-user ec2-user    4096 May 29 00:30 Neptune


In [48]:
import json
import ssl
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.aiohttp.transport import AiohttpTransport

# Neptune endpoint setup
NEPTUNE_ENDPOINT = "wss://db-neptune-1.cluster-cuzxvxpbce3r.us-east-1.neptune.amazonaws.com:8182/gremlin"
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE
transport = AiohttpTransport(ssl_context=ssl_context)

# Graph connection
graph = Graph()
connection = DriverRemoteConnection(NEPTUNE_ENDPOINT, "g", transport_factory=lambda: transport)
g = graph.traversal().withRemote(connection)

# Clear existing graph (optional)
try:
    g.V().drop().toList()
except Exception as e:
    print(f"Graph clear error: {e}")

In [49]:
import json
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
import ssl
import xml.etree.ElementTree as ET

# Neptune connection
NEPTUNE_ENDPOINT = "wss://db-neptune-1.cluster-cuzxvxpbce3r.us-east-1.neptune.amazonaws.com:8182/gremlin"
ssl_ctx = ssl.create_default_context()
ssl_ctx.check_hostname = False
ssl_ctx.verify_mode = ssl.CERT_NONE
transport = AiohttpTransport(ssl_context=ssl_ctx)
connection = DriverRemoteConnection(NEPTUNE_ENDPOINT, "g", transport_factory=lambda: transport)
graph = Graph()
g = graph.traversal().withRemote(connection)

# Clear graph
try: g.V().drop().toList()
except: print("Graph clear failed or already empty")

# Load and process facts
with open("amzn_facts.json") as f:
    data = json.load(f)

# Adjust depending on real structure
if isinstance(data, dict) and "factList" in data and isinstance(data["factList"], list):
    facts = data["factList"]
elif isinstance(data, list):
    facts = data
else:
    raise ValueError("Unknown structure in amzn_facts.json")

fact_nodes = {}
count = 0
for i, fact in enumerate(facts):
    if isinstance(fact, list) and len(fact) >= 2:
        concept = str(fact[0])
        value = str(fact[1])
        node = g.addV("Fact").property("id", concept).property("value", value).next()
        fact_nodes[concept] = node
        count += 1

print(f"✅ Loaded {count} facts")


✅ Loaded 1499 facts


In [51]:
import json
import ssl
import xml.etree.ElementTree as ET
from gremlin_python.structure.graph import Graph
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.graph_traversal import __

# Neptune setup
NEPTUNE_ENDPOINT = "wss://db-neptune-1.cluster-cuzxvxpbce3r.us-east-1.neptune.amazonaws.com:8182/gremlin"
ssl_ctx = ssl.create_default_context()
ssl_ctx.check_hostname = False
ssl_ctx.verify_mode = ssl.CERT_NONE
transport = AiohttpTransport(ssl_context=ssl_ctx)
connection = DriverRemoteConnection(NEPTUNE_ENDPOINT, "g", transport_factory=lambda: transport)
graph = Graph()
g = graph.traversal().withRemote(connection)

# Clear graph
try:
    g.V().drop().toList()
    print("✅ Graph cleared")
except Exception as e:
    print("⚠️ Graph clear failed:", e)

# Load JSON facts
with open("amzn_facts.json") as f:
    data = json.load(f)

facts = data.get("factList", [])
fact_nodes = {}
label_to_concept = {}
temporal_map = {}

# Create fact nodes
for i, fact in enumerate(facts):
    if isinstance(fact, list) and len(fact) == 3:
        name = fact[1].get("name")
        value = fact[2].get("value")
        context = fact[2].get("contextRef")
        label = fact[2].get("label", name)

        if name and value:
            uid = f"{name}_{context}"
            g.addV("Fact")\
                .property("id", uid)\
                .property("name", name)\
                .property("value", value)\
                .property("context", context)\
                .property("label", label)\
                .next()

            fact_nodes[uid] = uid
            label_to_concept[f"loc_{name}"] = uid
            temporal_map.setdefault(name, []).append((context, uid))

print("📊 Facts parsed:", len(facts))
print("✅ Nodes inserted:", len(fact_nodes))

# Parse arcs from linkbases
def parse_arcs(xml_path, arc_tag, edge_label):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    ns = {'link': 'http://www.xbrl.org/2003/linkbase', 'xlink': 'http://www.w3.org/1999/xlink'}

    label_map = {}
    for loc in root.findall(".//link:loc", ns):
        label = loc.attrib.get('{http://www.w3.org/1999/xlink}label')
        href = loc.attrib.get('{http://www.w3.org/1999/xlink}href')
        if label and href:
            concept = href.split("#")[-1]
            uid = label_to_concept.get(f"loc_{concept}")
            if uid:
                label_map[label] = uid

    edge_count = 0
    for arc in root.findall(f".//link:{arc_tag}", ns):
        from_label = arc.attrib.get('{http://www.w3.org/1999/xlink}from')
        to_label = arc.attrib.get('{http://www.w3.org/1999/xlink}to')
        from_id = label_map.get(from_label)
        to_id = label_map.get(to_label)

        if from_id in fact_nodes and to_id in fact_nodes:
            try:
                g.V().has("id", from_id).addE(edge_label).to(__.V().has("id", to_id)).next()
                edge_count += 1
            except:
                continue

    print(f"✅ {edge_label} edges added:", edge_count)

# Linkbase parsing
parse_arcs("amzn-20241231_cal.xml", "calculationArc", "calculation")
parse_arcs("amzn-20241231_def.xml", "definitionArc", "definition")
parse_arcs("amzn-20241231_lab.xml", "labelArc", "label")
parse_arcs("amzn-20241231_pre.xml", "presentationArc", "presentation")

# Add temporal links
temporal_links = 0
for concept, entries in temporal_map.items():
    entries.sort()  # sort by contextRef
    for i in range(len(entries) - 1):
        from_uid = entries[i][1]
        to_uid = entries[i + 1][1]
        if from_uid in fact_nodes and to_uid in fact_nodes:
            try:
                g.V().has("id", from_uid).addE("temporal").to(__.V().has("id", to_uid)).next()
                temporal_links += 1
            except:
                continue

print("✅ Temporal edges added:", temporal_links)
print("✅ Graph build complete")


✅ Graph cleared
📊 Facts parsed: 1499
✅ Nodes inserted: 1372
✅ calculation edges added: 0
✅ definition edges added: 0
✅ label edges added: 0
✅ presentation edges added: 0
✅ Temporal edges added: 967
✅ Graph build complete


In [52]:
my_node_labels = '{"Fact":"id"}'  # assumes 'Fact' vertices have an 'id' property
my_edge_labels = '{"calculation":"label", "definition":"label", "label":"label", "presentation":"label", "nextYear":"label"}'


In [53]:
%%gremlin -d id -g id -de label -l 20
g.V().hasLabel('Fact').outE("temporal").inV().path().by(elementMap()).limit(20)
